In [8]:
import pandas as pd
import os
from pathlib import Path

def get_image_names_as_dataframe(folder_path):
    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
    image_data = []

    folder = Path(folder_path)
    try:
        for subfolder in folder.iterdir():
            if subfolder.is_dir():
                for filename in os.listdir(subfolder):
                    if filename.lower().endswith(image_extensions):
                        class_name = subfolder.name
                        image_id = Path(filename).stem
                        image_data.append({'ImageName': f"{class_name}/{image_id}"})
        return pd.DataFrame(image_data)
    except FileNotFoundError:
        print(f"Error: Folder '{folder}' not found.")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

# Example usage:
folder_path = r"C:\Users\vihan\Desktop\Dataset"
image_df = get_image_names_as_dataframe(folder_path)
print(image_df)

          ImageName
0        beatroot/1
1       beatroot/10
2      beatroot/100
3       beatroot/11
4       beatroot/12
...             ...
22383  white_rice/5
22384  white_rice/6
22385  white_rice/7
22386  white_rice/8
22387  white_rice/9

[22388 rows x 1 columns]


In [ ]:
# Save image names to a text file (one per line)
#image_df.to_csv('image_names_new.txt', index=False, header=False)

In [14]:
from sklearn.model_selection import train_test_split

# Get the list of image names
image_names = image_df['ImageName'].tolist()

# Split into 80% train, 20% test (adjust test_size as needed)
train_names, test_names = train_test_split(image_names, test_size=0.2, random_state=42, shuffle=True)

# Save to text files
with open('train_names.txt', 'w') as f:
    for name in train_names:
        f.write(f"{name}\n")

with open('test_names.txt', 'w') as f:
    for name in test_names:
        f.write(f"{name}\n")

In [5]:
# Read the file and extract class names
with open(r"C:\Users\vihan\projects\final-one\models\image_names_new.txt", "r") as f:
    lines = f.read().splitlines()

# Extract class names (the part before '/')
class_names = [line.split('/')[0] for line in lines]

# Get unique class names
unique_class_names = sorted(set(class_names))

print(unique_class_names)

['beatroot', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_wings', 'deviled_eggs', 'dhal', 'donuts', 'eggs_benedict', 'fish_and_chips', 'french_fries', 'fried_egg', 'grilled_salmon', 'hot_dog', 'ice_cream', 'kottu', 'lasagna', 'macaroni_and_cheese', 'omelette', 'pork_chop', 'red_rice', 'samosa', 'steak', 'sushi', 'white_rice']


In [6]:
# Write unique class names to a text file, one per line
with open(r"C:\Users\vihan\projects\final-one\models\classes_new.txt", "w") as f:
    for name in unique_class_names:
        f.write(f"{name}\n")

print("Saved unique class names to classes.txt")

Saved unique class names to classes.txt
